In [78]:
import FirstCycling as FC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [79]:
fc = FC.FirstCycling()

In [80]:
fc.get_calendar(year=2021, month=3, cat='WT')

No race for this category in this month


In [81]:
teams =fc.get_teams(year=[2022,2021], cat=['WT','Junior','WWT'])

In [82]:
display(teams)

,team,country,bike,wt_wins,uci_wins,uci_ranking,gender,year,category,team_url,team_id
0,AG2R Citroën Team,France,BMC,0,0,10,M,2022,WT,https://firstcycling.com/team.php?l=21801,21801
1,Astana Qazaqstan Team,Kazakhstan,Wilier,0,1,18,M,2022,WT,https://firstcycling.com/team.php?l=21803,21803
2,Bahrain Victorious,Bahrain,Merida,1,4,9,M,2022,WT,https://firstcycling.com/team.php?l=21804,21804
3,BORA-hansgrohe,Germany,Specialized,0,5,8,M,2022,WT,https://firstcycling.com/team.php?l=22245,22245
4,Cofidis,France,De Rosa,0,4,12,M,2022,WT,https://firstcycling.com/team.php?l=22290,22290
...,...,...,...,...,...,...,...,...,...,...,...
963,Movistar Team,Spain,Canyon,7,22,2,W,2021,WWT,https://firstcycling.com/team.php?l=21875,21875
964,Team BikeExchange,Australia,Bianchi,2,7,10,W,2021,WWT,https://firstcycling.com/team.php?l=21874,21874
965,Team DSM,Germany,Scott,3,18,4,W,2021,WWT,https://firstcycling.com/team.php?l=21876,21876
966,Team SD Worx,Netherlands,Specialized,11,33,1,W,2021,WWT,https://firstcycling.com/team.php?l=22010,22010


In [84]:
fc.get_riders(21804)

AttributeError: 'FirstCycling' object has no attribute 'get_riders'

In [63]:
team_url = "https://firstcycling.com/team.php?l=21804&riders=2"
soup = BeautifulSoup(requests.get(team_url).text,'lxml')
tables = soup.find_all('table', class_='sortTabell')
for table in tables:
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    names = []
    age =[]
    nationalities=[]
    racedays =[]
    racekm=[]
    riders_id = []
    
    #get the riders' ID from the html file parsing
    riders_href =list(soup.select('a[href^="rider.php"]'))
    tags =[]
    for tag in riders_href:
        tags.append( str(tag) )
    for tag in tags:
        start = tag.find('rider.php?r=')+len('rider.php?r=')
        end = tag.find('&amp')
        riders_id.append(tag[start:end])
    
    #get riders info from the html
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        names.append(cols[0])
        age.append(cols[2])
        nationalities.append(cols[3])
        racedays.append(cols[8])
        racekm.append(cols[9])
df_riders = pd.DataFrame(list(zip(names, age, nationalities,racedays,racekm, riders_id)),
               columns =['Name', 'Age', 'Nationality', 'Race Days', 'Race Km','ID'])


display(Riders)

['163', '16816', '4863', '81835', '1006', '2338', '9014', '6555', '19784', '120', '2278', '65580', '115016', '80489', '16687', '37259', '27210', '73467', '55807', '929', '69317', '26', '6414', '8068', '3050', '37376', '63055', '97996']


,Name,Age,Nationality,Race Days,Race Km,ID
0,"Arashiro, Yukiya",37,Japan,5,778,163
1,"Bauhaus, Phil",27,Germany,8,1.326,16816
2,"Bilbao, Pello",32,Spain,20,3.154,4863
3,"Buitrago, Santiago",22,Colombia,5,831,81835
4,"Caruso, Damiano",34,Italy,12,1.950,1006
5,"Colbrelli, Sonny",31,Italy,3,558,2338
6,"Feng, Chun Kai",33,Taiwan,,,9014
7,"Gradek, Kamil",31,Poland,6,1.035,6555
8,"Haig, Jack",28,Australia,13,2.014,19784
9,"Haussler, Heinrich",38,Australia,9,1.530,120


In [ ]:
for id in teams['team_id']:
    full_url = team_url
        